# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 
model = mobilenet(num_classes=10, large_img=False)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [4]:
# Implement validation
def train(epoch):
    model.train()
    writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            writer.add_scalar('Loss/Loss', loss.item(), epoch)
            writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [5]:
def validate(epoch):
    model.eval()
    writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [9]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [7]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [8]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
writer = SummaryWriter()
writer.export_scalars_to_json("./all_scalars.json")

writer.close()

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.280370, Accuracy: 17.19
Train Epoch: 0 [1600/50000 (4%)]	Loss: 2.213747, Accuracy: 12.50
Train Epoch: 0 [3200/50000 (7%)]	Loss: 2.179886, Accuracy: 17.19
Train Epoch: 0 [4800/50000 (11%)]	Loss: 2.007280, Accuracy: 18.75
Train Epoch: 0 [6400/50000 (14%)]	Loss: 2.106038, Accuracy: 23.44
Train Epoch: 0 [8000/50000 (18%)]	Loss: 1.944879, Accuracy: 23.44
Train Epoch: 0 [9600/50000 (21%)]	Loss: 1.915234, Accuracy: 23.44
Train Epoch: 0 [11200/50000 (25%)]	Loss: 1.865076, Accuracy: 31.25
Train Epoch: 0 [12800/50000 (28%)]	Loss: 2.032271, Accuracy: 20.31
Train Epoch: 0 [14400/50000 (32%)]	Loss: 1.999610, Accuracy: 26.56
Train Epoch: 0 [16000/50000 (36%)]	Loss: 1.933717, Accuracy: 20.31
Train Epoch: 0 [17600/50000 (39%)]	Loss: 1.743079, Accuracy: 23.44
Train Epoch: 0 [19200/50000 (43%)]	Loss: 1.921615, Accuracy: 25.00
Train Epoch: 0 [20800/50000 (46%)]	Loss: 1.922472, Accuracy: 23.44
Train Epoch: 0 [22400/50000 (50%)]	Loss: 1.838185, Accuracy: 28.12
Train Ep

Train Epoch: 4 [1600/50000 (4%)]	Loss: 1.180426, Accuracy: 54.69
Train Epoch: 4 [3200/50000 (7%)]	Loss: 1.320766, Accuracy: 45.31
Train Epoch: 4 [4800/50000 (11%)]	Loss: 1.202568, Accuracy: 60.94
Train Epoch: 4 [6400/50000 (14%)]	Loss: 1.090907, Accuracy: 62.50
Train Epoch: 4 [8000/50000 (18%)]	Loss: 1.251675, Accuracy: 50.00
Train Epoch: 4 [9600/50000 (21%)]	Loss: 0.971748, Accuracy: 62.50
Train Epoch: 4 [11200/50000 (25%)]	Loss: 1.117808, Accuracy: 57.81
Train Epoch: 4 [12800/50000 (28%)]	Loss: 1.317500, Accuracy: 57.81
Train Epoch: 4 [14400/50000 (32%)]	Loss: 1.567803, Accuracy: 43.75
Train Epoch: 4 [16000/50000 (36%)]	Loss: 1.107582, Accuracy: 59.38
Train Epoch: 4 [17600/50000 (39%)]	Loss: 0.971159, Accuracy: 62.50
Train Epoch: 4 [19200/50000 (43%)]	Loss: 1.101761, Accuracy: 64.06
Train Epoch: 4 [20800/50000 (46%)]	Loss: 0.970010, Accuracy: 65.62
Train Epoch: 4 [22400/50000 (50%)]	Loss: 1.106820, Accuracy: 57.81
Train Epoch: 4 [24000/50000 (53%)]	Loss: 0.864458, Accuracy: 65.62
Tra

Train Epoch: 8 [3200/50000 (7%)]	Loss: 0.636818, Accuracy: 75.00
Train Epoch: 8 [4800/50000 (11%)]	Loss: 0.843927, Accuracy: 70.31
Train Epoch: 8 [6400/50000 (14%)]	Loss: 0.967177, Accuracy: 71.88
Train Epoch: 8 [8000/50000 (18%)]	Loss: 0.983967, Accuracy: 68.75
Train Epoch: 8 [9600/50000 (21%)]	Loss: 1.005182, Accuracy: 65.62
Train Epoch: 8 [11200/50000 (25%)]	Loss: 0.892029, Accuracy: 67.19
Train Epoch: 8 [12800/50000 (28%)]	Loss: 0.710518, Accuracy: 70.31
Train Epoch: 8 [14400/50000 (32%)]	Loss: 0.783861, Accuracy: 75.00
Train Epoch: 8 [16000/50000 (36%)]	Loss: 0.785201, Accuracy: 73.44
Train Epoch: 8 [17600/50000 (39%)]	Loss: 0.812930, Accuracy: 67.19
Train Epoch: 8 [19200/50000 (43%)]	Loss: 1.133851, Accuracy: 60.94
Train Epoch: 8 [20800/50000 (46%)]	Loss: 0.885585, Accuracy: 76.56
Train Epoch: 8 [22400/50000 (50%)]	Loss: 0.680017, Accuracy: 79.69
Train Epoch: 8 [24000/50000 (53%)]	Loss: 0.820649, Accuracy: 70.31
Train Epoch: 8 [25600/50000 (57%)]	Loss: 0.974587, Accuracy: 64.06
T

Train Epoch: 12 [3200/50000 (7%)]	Loss: 0.968002, Accuracy: 71.88
Train Epoch: 12 [4800/50000 (11%)]	Loss: 0.494100, Accuracy: 79.69
Train Epoch: 12 [6400/50000 (14%)]	Loss: 0.706615, Accuracy: 68.75
Train Epoch: 12 [8000/50000 (18%)]	Loss: 0.653209, Accuracy: 81.25
Train Epoch: 12 [9600/50000 (21%)]	Loss: 0.574666, Accuracy: 75.00
Train Epoch: 12 [11200/50000 (25%)]	Loss: 0.532650, Accuracy: 76.56
Train Epoch: 12 [12800/50000 (28%)]	Loss: 0.472055, Accuracy: 84.38
Train Epoch: 12 [14400/50000 (32%)]	Loss: 0.646320, Accuracy: 84.38
Train Epoch: 12 [16000/50000 (36%)]	Loss: 0.584054, Accuracy: 79.69
Train Epoch: 12 [17600/50000 (39%)]	Loss: 0.594443, Accuracy: 79.69
Train Epoch: 12 [19200/50000 (43%)]	Loss: 0.392108, Accuracy: 84.38
Train Epoch: 12 [20800/50000 (46%)]	Loss: 0.774863, Accuracy: 75.00
Train Epoch: 12 [22400/50000 (50%)]	Loss: 0.698894, Accuracy: 79.69
Train Epoch: 12 [24000/50000 (53%)]	Loss: 0.550816, Accuracy: 75.00
Train Epoch: 12 [25600/50000 (57%)]	Loss: 0.532657, Ac

Train Epoch: 16 [1600/50000 (4%)]	Loss: 0.584195, Accuracy: 76.56
Train Epoch: 16 [3200/50000 (7%)]	Loss: 0.557765, Accuracy: 78.12
Train Epoch: 16 [4800/50000 (11%)]	Loss: 0.634993, Accuracy: 76.56
Train Epoch: 16 [6400/50000 (14%)]	Loss: 0.686978, Accuracy: 76.56
Train Epoch: 16 [8000/50000 (18%)]	Loss: 0.454765, Accuracy: 84.38
Train Epoch: 16 [9600/50000 (21%)]	Loss: 0.455338, Accuracy: 84.38
Train Epoch: 16 [11200/50000 (25%)]	Loss: 0.679651, Accuracy: 75.00
Train Epoch: 16 [12800/50000 (28%)]	Loss: 0.474683, Accuracy: 81.25
Train Epoch: 16 [14400/50000 (32%)]	Loss: 0.634799, Accuracy: 79.69
Train Epoch: 16 [16000/50000 (36%)]	Loss: 0.725787, Accuracy: 75.00
Train Epoch: 16 [17600/50000 (39%)]	Loss: 0.572306, Accuracy: 78.12
Train Epoch: 16 [19200/50000 (43%)]	Loss: 0.534170, Accuracy: 81.25
Train Epoch: 16 [20800/50000 (46%)]	Loss: 0.580013, Accuracy: 76.56
Train Epoch: 16 [22400/50000 (50%)]	Loss: 0.667601, Accuracy: 76.56
Train Epoch: 16 [24000/50000 (53%)]	Loss: 0.754036, Accu

...

Train Epoch: 40 [1600/50000 (4%)]	Loss: 0.278283, Accuracy: 85.94
Train Epoch: 40 [3200/50000 (7%)]	Loss: 0.321921, Accuracy: 89.06
Train Epoch: 40 [4800/50000 (11%)]	Loss: 0.264950, Accuracy: 92.19
Train Epoch: 40 [6400/50000 (14%)]	Loss: 0.228238, Accuracy: 92.19
Train Epoch: 40 [8000/50000 (18%)]	Loss: 0.291482, Accuracy: 87.50
Train Epoch: 40 [9600/50000 (21%)]	Loss: 0.492357, Accuracy: 81.25
Train Epoch: 40 [11200/50000 (25%)]	Loss: 0.251545, Accuracy: 92.19
Train Epoch: 40 [12800/50000 (28%)]	Loss: 0.307002, Accuracy: 87.50
Train Epoch: 40 [14400/50000 (32%)]	Loss: 0.245996, Accuracy: 90.62
Train Epoch: 40 [16000/50000 (36%)]	Loss: 0.345007, Accuracy: 84.38
Train Epoch: 40 [17600/50000 (39%)]	Loss: 0.410488, Accuracy: 84.38
Train Epoch: 40 [19200/50000 (43%)]	Loss: 0.281661, Accuracy: 89.06
Train Epoch: 40 [20800/50000 (46%)]	Loss: 0.258283, Accuracy: 92.19
Train Epoch: 40 [22400/50000 (50%)]	Loss: 0.423369, Accuracy: 82.81
Train Epoch: 40 [24000/50000 (53%)]	Loss: 0.231547, Accu

Train Epoch: 44 [1600/50000 (4%)]	Loss: 0.331208, Accuracy: 90.62
Train Epoch: 44 [3200/50000 (7%)]	Loss: 0.240018, Accuracy: 89.06
Train Epoch: 44 [4800/50000 (11%)]	Loss: 0.192312, Accuracy: 89.06
Train Epoch: 44 [6400/50000 (14%)]	Loss: 0.210324, Accuracy: 93.75
Train Epoch: 44 [8000/50000 (18%)]	Loss: 0.146674, Accuracy: 98.44
Train Epoch: 44 [9600/50000 (21%)]	Loss: 0.152760, Accuracy: 93.75
Train Epoch: 44 [11200/50000 (25%)]	Loss: 0.289859, Accuracy: 90.62
Train Epoch: 44 [12800/50000 (28%)]	Loss: 0.274454, Accuracy: 89.06
Train Epoch: 44 [14400/50000 (32%)]	Loss: 0.258937, Accuracy: 92.19
Train Epoch: 44 [16000/50000 (36%)]	Loss: 0.142554, Accuracy: 95.31
Train Epoch: 44 [17600/50000 (39%)]	Loss: 0.211419, Accuracy: 95.31
Train Epoch: 44 [19200/50000 (43%)]	Loss: 0.165721, Accuracy: 93.75
Train Epoch: 44 [20800/50000 (46%)]	Loss: 0.368062, Accuracy: 85.94
Train Epoch: 44 [22400/50000 (50%)]	Loss: 0.141334, Accuracy: 92.19
Train Epoch: 44 [24000/50000 (53%)]	Loss: 0.208303, Accu

Train Epoch: 48 [1600/50000 (4%)]	Loss: 0.176564, Accuracy: 96.88
Train Epoch: 48 [3200/50000 (7%)]	Loss: 0.485929, Accuracy: 82.81
Train Epoch: 48 [4800/50000 (11%)]	Loss: 0.197909, Accuracy: 92.19
Train Epoch: 48 [6400/50000 (14%)]	Loss: 0.406307, Accuracy: 85.94
Train Epoch: 48 [8000/50000 (18%)]	Loss: 0.239648, Accuracy: 93.75
Train Epoch: 48 [9600/50000 (21%)]	Loss: 0.486013, Accuracy: 82.81
Train Epoch: 48 [11200/50000 (25%)]	Loss: 0.224905, Accuracy: 93.75
Train Epoch: 48 [12800/50000 (28%)]	Loss: 0.229877, Accuracy: 90.62
Train Epoch: 48 [14400/50000 (32%)]	Loss: 0.266384, Accuracy: 85.94
Train Epoch: 48 [16000/50000 (36%)]	Loss: 0.210965, Accuracy: 90.62
Train Epoch: 48 [17600/50000 (39%)]	Loss: 0.221407, Accuracy: 92.19
Train Epoch: 48 [19200/50000 (43%)]	Loss: 0.395301, Accuracy: 82.81
Train Epoch: 48 [20800/50000 (46%)]	Loss: 0.121348, Accuracy: 95.31
Train Epoch: 48 [22400/50000 (50%)]	Loss: 0.229926, Accuracy: 92.19
Train Epoch: 48 [24000/50000 (53%)]	Loss: 0.405179, Accu

Train Epoch: 52 [1600/50000 (4%)]	Loss: 0.250230, Accuracy: 85.94
Train Epoch: 52 [3200/50000 (7%)]	Loss: 0.262543, Accuracy: 85.94
Train Epoch: 52 [4800/50000 (11%)]	Loss: 0.181285, Accuracy: 93.75
Train Epoch: 52 [6400/50000 (14%)]	Loss: 0.324078, Accuracy: 90.62
Train Epoch: 52 [8000/50000 (18%)]	Loss: 0.237508, Accuracy: 92.19
Train Epoch: 52 [9600/50000 (21%)]	Loss: 0.151526, Accuracy: 96.88
Train Epoch: 52 [11200/50000 (25%)]	Loss: 0.304031, Accuracy: 89.06
Train Epoch: 52 [12800/50000 (28%)]	Loss: 0.201387, Accuracy: 92.19
Train Epoch: 52 [14400/50000 (32%)]	Loss: 0.178091, Accuracy: 90.62
Train Epoch: 52 [16000/50000 (36%)]	Loss: 0.212224, Accuracy: 92.19
Train Epoch: 52 [17600/50000 (39%)]	Loss: 0.208206, Accuracy: 89.06
Train Epoch: 52 [19200/50000 (43%)]	Loss: 0.131575, Accuracy: 95.31
Train Epoch: 52 [20800/50000 (46%)]	Loss: 0.282246, Accuracy: 90.62
Train Epoch: 52 [22400/50000 (50%)]	Loss: 0.269572, Accuracy: 89.06
Train Epoch: 52 [24000/50000 (53%)]	Loss: 0.081217, Accu

Train Epoch: 56 [1600/50000 (4%)]	Loss: 0.334675, Accuracy: 89.06
Train Epoch: 56 [3200/50000 (7%)]	Loss: 0.164946, Accuracy: 93.75
Train Epoch: 56 [4800/50000 (11%)]	Loss: 0.208952, Accuracy: 90.62
Train Epoch: 56 [6400/50000 (14%)]	Loss: 0.242767, Accuracy: 92.19
Train Epoch: 56 [8000/50000 (18%)]	Loss: 0.148602, Accuracy: 93.75
Train Epoch: 56 [9600/50000 (21%)]	Loss: 0.249775, Accuracy: 92.19
Train Epoch: 56 [11200/50000 (25%)]	Loss: 0.369644, Accuracy: 84.38
Train Epoch: 56 [12800/50000 (28%)]	Loss: 0.284034, Accuracy: 90.62
Train Epoch: 56 [14400/50000 (32%)]	Loss: 0.385432, Accuracy: 82.81
Train Epoch: 56 [16000/50000 (36%)]	Loss: 0.163530, Accuracy: 92.19
Train Epoch: 56 [17600/50000 (39%)]	Loss: 0.320190, Accuracy: 90.62
Train Epoch: 56 [19200/50000 (43%)]	Loss: 0.429660, Accuracy: 85.94
Train Epoch: 56 [20800/50000 (46%)]	Loss: 0.243774, Accuracy: 87.50
Train Epoch: 56 [22400/50000 (50%)]	Loss: 0.152424, Accuracy: 93.75
Train Epoch: 56 [24000/50000 (53%)]	Loss: 0.206841, Accu

# Step 3: Test

In [10]:
test(epoch)


Test set: Average loss: 0.5272, Accuracy: 8457/10000 (84.57%)

